In [ ]:
# 훈련과 테스트에서 다르게 동작하는 층

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

input_shape = X_train_scaled.shape[1:]

In [2]:
class AddGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev

    def call(self, X, training=None):
        if training:
            noise = tf.random.normal(tf.shape(X), stddev=self.stddev)
            return X + noise
        else:
            return X

    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

In [3]:
model = keras.models.Sequential([
    AddGaussianNoise(stddev=1.0),
    keras.layers.Dense(30, activation="selu"),
    keras.layers.Dense(1)
])

In [4]:
model.compile(loss="mse", optimizer="nadam")
model.fit(X_train_scaled, y_train, epochs=10,
          validation_data=(X_valid_scaled, y_valid))
model.evaluate(X_test_scaled, y_test)

Epoch 1/10
363/363 [==============================] - 9s 7ms/step - loss: 2.5715 - val_loss: 6.1746
Epoch 2/10
363/363 [==============================] - 2s 6ms/step - loss: 1.0429 - val_loss: 3.1161
Epoch 3/10
363/363 [==============================] - 2s 6ms/step - loss: 1.0171 - val_loss: 1.4232
Epoch 4/10
363/363 [==============================] - 2s 7ms/step - loss: 0.9993 - val_loss: 0.9983
Epoch 5/10
363/363 [==============================] - 2s 5ms/step - loss: 0.9916 - val_loss: 0.9445
Epoch 6/10
363/363 [==============================] - 1s 3ms/step - loss: 0.9922 - val_loss: 0.9481
Epoch 7/10
363/363 [==============================] - 1s 3ms/step - loss: 0.9842 - val_loss: 0.8566
Epoch 8/10
363/363 [==============================] - 1s 3ms/step - loss: 0.9886 - val_loss: 0.8463
Epoch 9/10
363/363 [==============================] - 1s 4ms/step - loss: 0.9690 - val_loss: 0.9226
Epoch 10/10
162/162 [==============================] - 0s 2ms/step - loss: 0.7711


0.7710556983947754

In [ ]:
# 사용자 정의 모델

In [5]:
X_new_scaled = X_test_scaled

In [ ]:
# 1. ResidualBlock 층 만들기

In [6]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons, activation="elu",
                                          kernel_initializer="he_normal")
                       for _ in range(n_layers)]

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z


In [ ]:
# 2. 서브클래싱 API로 모델을 정의
# 3번 반복 vs 3개 층 -> 층이 3개면 파라미터도 3배

In [7]:
class ResidualRegressor(keras.models.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30, activation="elu",
                                          kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = keras.layers.Dense(output_dim)

    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

In [9]:
model = ResidualRegressor(1)   # output_dim = 1
model.compile(loss="mse", optimizer="nadam")
history = model.fit(X_train_scaled, y_train, epochs=5)
score = model.evaluate(X_test_scaled, y_test)
y_pred = model.predict(X_new_scaled)

Epoch 1/5
363/363 [==============================] - 4s 6ms/step - loss: 17.4462
Epoch 2/5
363/363 [==============================] - 2s 4ms/step - loss: 3.9160
Epoch 3/5
363/363 [==============================] - 2s 5ms/step - loss: 1.0849
Epoch 4/5
363/363 [==============================] - 2s 4ms/step - loss: 0.7958
Epoch 5/5
162/162 [==============================] - 0s 2ms/step - loss: 0.6089


In [ ]:
# 모델 구성 요소에 기반한 손실과 지표
# 새로운 규제

In [ ]:
# 예제 : 재구성 손실을 추가

In [14]:
class ReconstructingRegressor(keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(30, activation="selu",
                                          kernel_initializer="lecun_normal")
                       for _ in range(5)]
        self.out = keras.layers.Dense(output_dim)
        self.reconstruction_mean = keras.metrics.Mean(name="reconstruction_error")

    def build(self, batch_input_shape):
        n_inputs = batch_input_shape[-1]
        self.reconstruct = keras.layers.Dense(n_inputs)
        #super().build(batch_input_shape)

    def call(self, inputs, training=None):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        self.recon_loss = 0.05 * tf.reduce_mean(tf.square(reconstruction - inputs))
        self.add_loss(self.recon_loss)
        
        if training:
           result = self.reconstruction_mean(self.recon_loss)
           self.add_metric(result)
        return self.out(Z)
    
    def train_step(self, data):
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x)
            loss = self.compiled_loss(y, y_pred, regularization_losses=[self.recon_loss])

        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        return {m.name: m.result() for m in self.metrics}

In [15]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = ReconstructingRegressor(1)
model.compile(loss="mse", optimizer="nadam")
history = model.fit(X_train_scaled, y_train, epochs=2)
y_pred = model.predict(X_test_scaled)

Epoch 1/2
363/363 [==============================] - 3s 5ms/step - loss: 0.7886 - reconstruction_error: 0.0000e+00
Epoch 2/2
363/363 [==============================] - 2s 5ms/step - loss: 0.4134 - reconstruction_error: 0.0000e+00


In [12]:
print(y_pred)

[[0.87845206]
 [1.5451547 ]
 [3.9395456 ]
 ...
 [1.4461813 ]
 [2.4412837 ]
 [4.091943  ]]
